In [2]:
# determine percent changes 
# things to optimize
    # R value 
        # Roof 
        # Wall
        # Floor 
    # Window
        # Shgc 
        # U-Factor
    # Lighting 
    # Daylight sensors 
    # Equipment 
    # Setpoints -> consider pmv as a constraint 


# calculate energy savings 
# think about npv / payback period to determine max decent const cost 


# Imports

In [40]:
import numpy as np
import os
from eppy import *
from eppy.modeleditor import IDF
import pandas as pd
from ladybug.sql import SQLiteResult
from ladybug import analysisperiod as ap
from collections import OrderedDict
from eppy.results import fasthtml


In [4]:
opt_models_dir = "/Users/julietnwagwuume-ezeoke/Documents/cee256_local/_final_project_local/optimization/opt_models"


# set the idd
iddfile = "/Applications/OpenStudioApplication-1.1.1/EnergyPlus/Energy+.idd"
IDF.setiddname(iddfile)

# get the epw
epw = "/Users/julietnwagwuume-ezeoke/Documents/cee256_local/weather_files/CA_PALO-ALTO-AP_724937S_19.epw"

In [31]:
root_idf_path = "/Users/julietnwagwuume-ezeoke/Documents/cee256_local/_final_project_local/calibration/eppy_adjusted_models_2/03_09_test/in2.idf"
root_idf = IDF(root_idf_path, epw)

# Optim

In [ ]:
def change_window_shgc(idf0, pc):
    window_mat = [s  for s in idf0.idfobjects["WindowMaterial:SimpleGlazingSystem"] if s.Name == "Thornton Window Material 1"][0]
    # get current value 
    curr_val = window_mat.Solar_Heat_Gain_Coefficient 
    print(curr_val)
    # calculate new values 
    new_val = perc_change_val(pc, curr_val)
    # adjust the idf 
    window_mat.Solar_Heat_Gain_Coefficient = new_val
    # return the idf
    print([s  for s in idf0.idfobjects["WindowMaterial:SimpleGlazingSystem"] if s.Name == "Thornton Window Material 1"][0])
    return (idf0, new_val)

In [23]:
pc = -10
str_pc = str(pc)
if str_pc[0] == "-":
    str_pc = str_pc.replace("-", "neg_")
str_pc

'neg_10'

# Helper Functions

In [84]:
def perc_change_val(pc, xo):
    "given percent change pc, and xo, calc x_new, xn"
    xn = (xo*pc)/100 + xo
    xn_real = max(0, xn)
    return xn_real

def perc_change_arr(pc_arr, xo):
    "given array of percent changes pc_arr, and xo, calc x_new_arr, xn_arr"
    xn_arr = [perc_change_val(pc, xo) for pc in pc_arr]
    return xn_arr
        
# generate new idf 
def new_idfs(opt_group_name, idf_adjust_fx):
    # make a group folder
    group_folder = os.path.join(opt_models_dir, opt_group_name)
    os.makedirs(group_folder)
    # keep track of the values changed in the idf 
    new_vals = {}
    for ix, pc in enumerate(pc_arr):
        # make a folder for the specific run 
        # rename negatives 
        str_pc = str(pc)
        if str_pc[0] == "-":
            str_pc = str_pc.replace("-", "neg_")
        str_pc
        run_folder = os.path.join(group_folder, f"run_{str_pc}")
        os.makedirs(run_folder)
        # make a copy of an og idf
        idf0_path = os.path.join(run_folder, "opt.idf")
        root_idf.save(idf0_path)
        # initialize this new idf 
        idf0 = IDF(idf0_path, epw)

        # call function to modify the idf 
        idf0, new_val = idf_adjust_fx(idf0, pc)
        # save the idf here 
        idf0.save()
        # save the new vals
        new_vals[pc] = new_val
    print(group_folder)
    # save to csv in group folder 
    new_vals_df = pd.DataFrame.from_dict(new_vals, orient="index")
    new_vals_df.to_csv(os.path.join(group_folder, "values.csv"))
    return group_folder


def get_sub_dirs(group_path):
    sub_dirs = [d for d in os.listdir(group_path) if os.path.isdir(os.path.join(group_path, d))]
    return sub_dirs

# run idfs 
def run_idfs(group_path):
    print(group_path)
    sub_dirs = get_sub_dirs(group_path)
    for sub_dir in sub_dirs:
        dir = os.path.join(group_path, sub_dir)
        print(sub_dir)
        idf_path = os.path.join(dir, "opt.idf")
        idf0 = IDF(idf_path, epw)
        idf0.run(output_directory = dir)



def extract_data(group_path):
    "can execute after run_idfs to get the data"
    print(group_path)

    sub_dirs = get_sub_dirs(group_path)

    sub_dir_data = OrderedDict()
    for sub_dir in sub_dirs:
        dir = os.path.join(group_path, sub_dir)

        # energy 
        sql_path = os.path.join(dir, "eplusout.sql")
        sqld = SQLiteResult(sql_path)

        # costs 
        html_path = os.path.join(dir, "eplustbl.htm")
        filehandle = open(html_path, 'r') # get a file handle to the html file
        namedtable = fasthtml.tablebyname(filehandle, "Tariff Summary")
        table_rows = namedtable[1]

        data = OrderedDict({
            # energy kwh 
            "elect": sqld.data_collections_by_output_name("Electricity:Facility")[-1].total,
            "hot": sqld.data_collections_by_output_name("DistrictHeating:Facility")[-1].total,
            "chill": sqld.data_collections_by_output_name("DistrictCooling:Facility")[-1].total,
            # costs $
            "elect_c": [t for t in table_rows if t[0] == 'ELECTRICITY TARIFF'][0][-1],
            "hot_c": [t for t in table_rows if t[0] == 'DISTRICTHEATING TARIFF'][0][-1],
            "chill_c": [t for t in table_rows if t[0] == 'DISTRICTCOOLING TARIFF'][0][-1],
        })
        data["total"] = data["elect"] +  data["hot"] +  data["chill"] 
        data["total_c"] = data["elect_c"] +  data["hot_c"] +  data["chill_c"] 

        sub_dir_data[sub_dir] = data
    
    # dataframe with energy and cost data 
    df = pd.DataFrame.from_dict(sub_dir_data, orient="index")

    # add edited values 
    vals = pd.read_csv(os.path.join(group_path, "values.csv"))
    dict_val =  OrderedDict()
    for i, val in zip(list(df.index), vals['0']):
        dict_val[i] = val
    df["vals"] = dict_val.values()

    # save as csv 
    df.to_csv(os.path.join(group_path, "data.csv"))
    
    # add the edited values 
    return df, vals






# extract relevant data 
# create dictionary of runs and their names 
# pass to calibration functions like thing 
# just care about hot water, elect, chilled water, and their tarrifs
# inside folder, csv with the data 
# for a group, csv in group folder + csv in run folder with results -> another csv 
# all this in an executable 



# analysis notebook 
# then can correlate energy use with cost, 
# as well as tornado diagrams for energy use and cost -> subplots
# can compare individual schemes to see which give greatest e savings / cost 
# can pair schemes to maximize.. 




# Test

In [ ]:
pc_arr = [-10,10]
a = new_idfs("test_b0", change_window_shgc)
# run_idfs(a)